In [6]:
import numpy as np
import pandas as pd
import pylab
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib import cm

In [7]:
def load_data(X_filename, label_filename):
    print('load data.....')
    mnist_x = pd.read_csv(r"mnist_X.csv",header=None)
    mnist_y = pd.read_csv(r"mnist_label.csv",header=None)
    return mnist_x.values, mnist_y.values

In [8]:
def Hbeta(D=np.array([]), beta=1.0):
    """
        Compute the perplexity and the P-row for a specific value of the
        precision of a Gaussian distribution.
    """

    # Compute P-row and corresponding perplexity
    P = np.exp(-D.copy() * beta)
    sumP = sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P


def x2p(X=np.array([]), tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values in such a way that each
        conditional Gaussian has the same perplexity.
    """

    # Initialize some variables
    print("Computing pairwise distances...")
    (n, d) = X.shape
    sum_X = np.sum(np.square(X), 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    P = np.zeros((n, n))
    beta = np.ones((n, 1))
    logU = np.log(perplexity)

    # Loop over all datapoints
    for i in range(n):

        # Print progress
        if i % 500 == 0:
            print("Computing P-values for point %d of %d..." % (i, n))

        # Compute the Gaussian kernel and entropy for the current precision
        betamin = -np.inf
        betamax = np.inf
        Di = D[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
        (H, thisP) = Hbeta(Di, beta[i])

        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while np.abs(Hdiff) > tol and tries < 50:

            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2.
                else:
                    beta[i] = (beta[i] + betamax) / 2.
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2.
                else:
                    beta[i] = (beta[i] + betamin) / 2.

            # Recompute the values
            (H, thisP) = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1

        # Set the final row of P
        P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP

    # Return final P-matrix
    print("Mean value of sigma: %f" % np.mean(np.sqrt(1 / beta)))
    return P

In [9]:
def pca(X=np.array([]), no_dims=50):
    """
        Runs PCA on the NxD array X in order to reduce its dimensionality to
        no_dims dimensions.
    """

    print("Preprocessing the data using PCA...")
    (n, d) = X.shape
    X = X - np.tile(np.mean(X, 0), (n, 1))
    (l, M) = np.linalg.eig(np.dot(X.T, X))
    Y = np.dot(X, M[:, 0:no_dims])
    return Y


def ssne(X=np.array([]), no_dims=2, initial_dims=50, perplexity=30.0):
    """
        Runs S-SNE on the dataset in the NxD array X to reduce its
        dimensionality to no_dims dimensions. The syntaxis of the function is
        `Y = ssne.ssne(X, no_dims, perplexity), where X is an NxD NumPy array.
    """
    print ("SSNE")

    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array X should have type float.")
        return -1
    if round(no_dims) != no_dims:
        print("Error: number of dimensions should be an integer.")
        return -1

    # Initialize variables
    X = pca(X, initial_dims).real
    (n, d) = X.shape
    max_iter = 1000
    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 500
    min_gain = 0.01
    Y = np.random.randn(n, no_dims)
    dY = np.zeros((n, no_dims))
    iY = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))

    # Compute P-values
    P = x2p(X, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)
    P = P * 4.                                  # early exaggeration
    P = np.maximum(P, 1e-12)

    # Run iterations
    oldError = 0
    loss = []
    for iter in range(max_iter):

        # Compute pairwise affinities
        #sum_Y = np.sum(np.square(Y), 1)
        #num = -2. * np.dot(Y, Y.T)
        #num = np.exp(-(np.add(np.add(num, sum_Y).T, sum_Y)))
        num = np.exp(-cdist(Y, Y,'euclidean')**2)
        num[range(n), range(n)] = 0.
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)

        # Compute gradient
        PQ = P - Q
        for i in range(n):
            dY[i, :] =  np.sum(np.tile(PQ[:, i] , (no_dims, 1)).T * (Y[i, :] - Y), 0)

        # Perform the update
        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum
        gains = (gains + 0.2) * ((dY > 0.) != (iY > 0.)) + \
                (gains * 0.8) * ((dY > 0.) == (iY > 0.))
        gains[gains < min_gain] = min_gain
        iY = momentum * iY - eta * (gains * dY)
        Y = Y + iY
        Y = Y - np.tile(np.mean(Y, 0), (n, 1))

        # Compute current value of cost function
        if (iter + 1) % 100 == 0:
            C = np.sum(P * np.log(P / Q))
            print("Iteration %d: error is %f" % (iter + 1, C))
            loss.append (C)

        # Stop lying about P-values
        if iter == 100:
            P = P / 4.


    #print (loss)

    # Return solution
    return Y,P,Q

def argsort(seq):
    # http://stackoverflow.com/questions/3071415/efficient-method-to-calculate-the-rank-vector-of-a-list-in-python
    return sorted(range(len(seq)), key=seq.__getitem__)

In [10]:
if __name__ == "__main__":
    #print("Run Y = tsne.tsne(X, no_dims, perplexity) to perform t-SNE on your dataset.")
    #print("Running example on 2,500 MNIST digits...")
    X, labels = load_data('mnist_X.csv', 'mnist_label.csv')
    labels = labels.flatten()
    for per in range (3,6,1):
        perplexity = 2**per
        print ("perplexity ",perplexity)
        Y,P,Q = ssne(X, 2, 50, perplexity)
        
        data = [[[0 for i in range(2)] for j in range(0)] for k in range(5)]

        for i in range (5):
            idx = np.where(labels == i+1)
            data[i].append(Y[idx])
        plt.axis('equal')
        for i in range (5):
            print("i",i)
            plt.plot(np.squeeze(data[i])[:,0],np.squeeze(data[i])[:,1],marker='o',color=plt.cm.tab10(i+1),markersize=2,alpha =0.8,linestyle='none',label = str(i+1))
 
        plt.title("Symetric SNE with perplexity = " + str(perplexity))
        plt.legend(loc="best")
        plt.savefig("plots/SSNE"+str(perplexity)+".png", bbox_inches="tight",dpi=150)
        plt.clf()
        
        
        PFlatten = P.flatten()
        QFlatten = Q.flatten()
        plt.hist (PFlatten,bins=50,rwidth=1.0, log = True,density =True)
        plt.title("Histogram high-dimesion in log-scale")
        plt.savefig("plots/Hist_SSNE_P"+str(perplexity)+".png", bbox_inches="tight",dpi=150)
        plt.clf()
        plt.title("Histogram low-dimesion in log-scale")
        plt.hist (QFlatten,bins=50,rwidth=1.0, log = True,density =True)
        plt.savefig("plots/Hist_SSNE_Q"+str(perplexity)+".png", bbox_inches="tight",dpi=150)
        plt.clf()

load data.....
perplexity  8
SSNE
Preprocessing the data using PCA...
Computing pairwise distances...
Computing P-values for point 0 of 5000...
Computing P-values for point 500 of 5000...
Computing P-values for point 1000 of 5000...
Computing P-values for point 1500 of 5000...
Computing P-values for point 2000 of 5000...
Computing P-values for point 2500 of 5000...
Computing P-values for point 3000 of 5000...
Computing P-values for point 3500 of 5000...
Computing P-values for point 4000 of 5000...
Computing P-values for point 4500 of 5000...
Mean value of sigma: 1.508087
Iteration 100: error is 19.468866
Iteration 200: error is 2.667211
Iteration 300: error is 2.665757
Iteration 400: error is 2.664819
Iteration 500: error is 2.664598
Iteration 600: error is 2.664320
Iteration 700: error is 2.662109
Iteration 800: error is 2.662214
Iteration 900: error is 2.662252
Iteration 1000: error is 2.662258
i 0
i 1
i 2
i 3
i 4
perplexity  16
SSNE
Preprocessing the data using PCA...
Computing pair